In [1]:
%cd ../data

C:\Users\gawie\Desktop\my_files\projects\capstone\data


In [2]:
import pandas as pd
import sqlite3
import ast
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder

In [3]:
conn = sqlite3.connect('valorant.db')

matches_df = pd.read_sql_query("SELECT * FROM Matches", conn)
games_df = pd.read_sql_query("SELECT * FROM Games", conn)
game_rounds_df = pd.read_sql_query("SELECT * FROM Game_Rounds", conn)
game_scoreboard_df = pd.read_sql_query("SELECT * FROM Game_Scoreboard", conn)
conn.close()

In [4]:
game_rounds_df.dropna(inplace=True)

In [5]:
game_rounds_df.reset_index(inplace=True)

In [6]:
len(game_rounds_df)

24077

In [7]:
%cd ../code

C:\Users\gawie\Desktop\my_files\projects\capstone\code


### Creating DataFrame from Database

In [8]:
# create empty dataframe
df = pd.DataFrame()

# loop through each game
for game in range(len(game_rounds_df['RoundHistory'])):
    # try to create a dataframe from the RoundHistory column
    try:
        # create a dataframe from the RoundHistory column
        temp_df = pd.DataFrame(ast.literal_eval(game_rounds_df.iloc[game]['RoundHistory'])).T
        # add the GameID, Team1ID, and Team2ID columns
        temp_df['GameID'] = game_rounds_df['GameID'][game]
        temp_df['Team1ID']= game_rounds_df['Team1ID'][game]
        temp_df['Team2ID'] = game_rounds_df['Team2ID'][game]
        # if the first round score is 1-0, then the first team is Team1Abbr
        if int(temp_df['ScoreAfterRound'][1].split('-')[0]) == 1:
            temp_df['Team1Abbr']= temp_df['|RoundWinner'].unique()[0]
            # if there are two unique winners, then the second team is Team2Abbr
            if len(temp_df['RoundWinner'].unique())==2:
                temp_df['Team2Abbr']= temp_df['RoundWinner'].unique()[1]
        # if the first round score is 0-1, then the first team is Team2Abbr
        else:
            temp_df['Team2Abbr']= temp_df['RoundWinner'].unique()[0]
            # if there are two unique winners, then the second team is Team1Abbr
            if len(temp_df['RoundWinner'].unique())==2:
                temp_df['Team1Abbr']= temp_df['RoundWinner'].unique()[1]
        # create a column for whether Team1 won the round
        temp_df['Team1Win'] = [1 if temp_df['Team1Abbr'][round+1] == temp_df['RoundWinner'][round+1] else 0 for round in range(len(temp_df['RoundWinner']))]
        # create columns for whether Team1 won the previous rounds
        temp_df['Team1WinLag1'] = temp_df['Team1Win'].shift(1)
        temp_df['Team1WinLag2'] = temp_df['Team1Win'].shift(2)
        temp_df['Team1WinLag3'] = temp_df['Team1Win'].shift(3)
        temp_df['Team1WinLag4'] = temp_df['Team1Win'].shift(4)
        temp_df['Team1WinLag5'] = temp_df['Team1Win'].shift(5)
        temp_df['Team1WinLag6'] = temp_df['Team1Win'].shift(6)
        temp_df['Team1WinLag7'] = temp_df['Team1Win'].shift(7)
        temp_df['Team1WinLag8'] = temp_df['Team1Win'].shift(8)
        # create columns for Team1's bank in the previous rounds
        temp_df['Team1BankLag1'] = temp_df['Team1Bank'].shift(1)
        temp_df['Team1BankLag2'] = temp_df['Team1Bank'].shift(2)
        temp_df['Team1BankLag3'] = temp_df['Team1Bank'].shift(3)
        temp_df['Team1BankLag4'] = temp_df['Team1Bank'].shift(4)
        temp_df['Team1BankLag5'] = temp_df['Team1Bank'].shift(5)
        temp_df['Team1BankLag6'] = temp_df['Team1Bank'].shift(6)
        temp_df['Team1BankLag7'] = temp_df['Team1Bank'].shift(7)
        temp_df['Team1BankLag8'] = temp_df['Team1Bank'].shift(8)
        # create columns for Team2's bank in the previous rounds
        temp_df['Team2BankLag1'] = temp_df['Team2Bank'].shift(1)
        temp_df['Team2BankLag2'] = temp_df['Team2Bank'].shift(2)
        temp_df['Team2BankLag3'] = temp_df['Team2Bank'].shift(3)
        temp_df['Team2BankLag4'] = temp_df['Team2Bank'].shift(4)
        temp_df['Team2BankLag5'] = temp_df['Team2Bank'].shift(5)
        temp_df['Team2BankLag6'] = temp_df['Team2Bank'].shift(6)
        temp_df['Team2BankLag7'] = temp_df['Team2Bank'].shift(7)
        temp_df['Team2BankLag8'] = temp_df['Team2Bank'].shift(8)         
        # create a column for the round number
        temp_df['RoundNo'] = [row+1 for row in range(len(temp_df))]
        # create a column for the difference in score between the two teams
        temp_df['RoundDiff'] = [int(temp_df['ScoreAfterRound'][row+1].split('-')[0]) - int(temp_df['ScoreAfterRound'][row+1].split('-')[1]) for row in range(len(temp_df)) if row < len(temp_df)]
        # shift the RoundDiff column up one row
        temp_df['RoundDiff'] = temp_df['RoundDiff'].shift(1).fillna(0)
        # add the temp_df to the df
        df = pd.concat([df, temp_df], ignore_index=True)
    # if there is a KeyError, then the game is not a full game
    except KeyError:
        pass

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
comps = pd.DataFrame(game_scoreboard_df[['GameID','TeamAbbreviation', 'Agent']].groupby(by=['GameID','TeamAbbreviation'])['Agent'].unique())

In [11]:
comps.dtypes

Agent    object
dtype: object

In [12]:
comps.reset_index(inplace=True)

In [13]:
df = df.astype({'GameID': 'object','Team1Abbr':'object'})

In [14]:
df = df.dropna(subset=['Team1Abbr', 'Team2Abbr'])

### Adding Team Composition to the DataFrame

In [15]:
# merge rounds_df with comps_df to get the agents for Team 1
rounds_comp = df.merge(comps,how='left', left_on=['GameID','Team1Abbr'], right_on=['GameID','TeamAbbreviation'])

# rename the agent column to team1Comp
rounds_comp.rename(columns={'Agent':'team1Comp'}, inplace=True)

# merge rounds_comp with comps_df to get the agents for Team 2
rounds_comp = rounds_comp.merge(comps,how='left', left_on=['GameID','Team2Abbr'], right_on=['GameID','TeamAbbreviation'])

# rename the agent column to team2Comp
rounds_comp.rename(columns={'Agent':'team2Comp'}, inplace=True)

# drop the TeamAbbreviation columns
rounds_comp = rounds_comp.drop(columns=['TeamAbbreviation_x', 'TeamAbbreviation_y'])

# merge rounds_comp with games_df to get the map for each game
rounds_comp = rounds_comp.merge(games_df[['GameID','Map']], how='left', on=['GameID'])

In [16]:
rounds_comp.dropna(subset=['team1Comp','team2Comp'], inplace=True)

In [17]:
# create a sparse matrix for the team2Comp column
mlb = MultiLabelBinarizer(sparse_output=True)

# join the sparse matrix to the rounds_comp dataframe
rounds_comp = rounds_comp.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(rounds_comp.pop('team2Comp')),
                index=rounds_comp.index,
                columns=['team2_' + x for x in mlb.classes_]))

In [18]:
mlb = MultiLabelBinarizer(sparse_output=True)
rounds_comp = rounds_comp.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(rounds_comp.pop('team1Comp')),
                index=rounds_comp.index,
                columns=['team1_' + x for x in mlb.classes_]))

In [19]:
#rounds_comp.to_csv('../data/round_comps.csv')